In [ ]:
# cpu_only_baseline.py
import numpy as np
import tensorflow as tf
from time import perf_counter

IMG_H, IMG_W, IMG_C = 28, 28, 1

# 1) Load test set
x_test = np.load('asl_test_images.npy').astype(np.float32)  # shape (N, 28, 28, 1)
y_test = np.load('asl_test_labels.npy').astype(np.int32)    # shape (N,)

# Normalize if needed
x_test /= 255.0

# 2) Load trained model
model = tf.keras.models.load_model('asl_cnn_full.h5')
model.summary()

# 3) Accuracy
logits = model.predict(x_test, batch_size=64)
y_pred = np.argmax(logits, axis=1)
acc = np.mean(y_pred == y_test) * 100.0
print(f"CPU baseline top-1 accuracy: {acc:.2f}%")

# 4) Latency / FPS with batch size 1
n_runs = 500
latencies = []

for i in range(n_runs):
    idx = np.random.randint(0, x_test.shape[0])
    sample = x_test[idx:idx+1]

    t0 = perf_counter()
    _ = model.predict(sample, batch_size=1, verbose=0)
    t1 = perf_counter()

    latencies.append((t1 - t0) * 1000.0)  # ms

latencies = np.array(latencies, dtype=np.float32)
mean_ms = float(latencies.mean())
std_ms  = float(latencies.std(ddof=1))
fps     = 1000.0 / mean_ms

print(f"CPU latency: mean={mean_ms:.3f} ms, std={std_ms:.3f} ms, FPS={fps:.2f}")

np.save('cpu_latency_ms.npy', latencies)
np.save('cpu_predictions.npy', y_pred)
